In [1]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.core import Document
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import StorageContext
from llama_index.core.query_pipeline import QueryPipeline
#from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.agent.openai import OpenAIAgent
#from llama_index.llms.openai_like import OpenAILike
from llama_index.core.tools import QueryEngineTool, ToolMetadata
import os
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import ReActAgent
from llama_index.agent.openai import OpenAIAgent
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
    VectorStoreIndex,
)
from llama_index.core import SummaryIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.node_parser import SentenceSplitter
import os
from tqdm.notebook import tqdm
import pickle
from pathlib import Path

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

# Retrieve the OpenAI API key from the environment
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
#print(OPENAI_API_KEY)

In [2]:
def read_folder(path):
    # Walk through all files in the given path and return a list of documents
    documents = []
    all_files = []
    for root, dirs, files in os.walk(path):
        all_files.append(files)
        for file in files:
            document_path = os.path.join(root, file)
            documents.append(SimpleDirectoryReader(input_files=[document_path]).load_data())
            

    indexes = []
    for i in documents:
        indexes.append(VectorStoreIndex.from_documents(documents=i))
        
    return indexes, files

In [3]:
indexes, files = read_folder('data/')

llm = OpenAI(model="gpt-3.5-turbo-0613")
#print(files)

#tools = create_engines_tools(indexes)

tools = [QueryEngineTool.from_defaults(
    indexes[i].as_query_engine(), name=f"engine_{i}", description=f"Provides information about {files[i]}"
) for i in range(len(indexes))]


agent = OpenAIAgent.from_tools(
    tools=tools,
    llm=llm,
    verbose=True
)

Failed to load file data\customer data - preffered car price.csv with error: 'utf-8' codec can't decode byte 0xc5 in position 8694: invalid continuation byte. Skipping...


In [4]:
agent.chat("What are the characteristics of Volkswagen ID.3 Pro").response

Added user message to memory: What are the characteristics of Volkswagen ID.3 Pro
=== Calling Function ===
Calling function: engine_2 with args: {
  "input": "Volkswagen ID.3 Pro"
}
Got output: The Volkswagen ID.3 Pro is a hatchback electric car with a range of 340 miles, rear-wheel drive, Type 2 CCS charging capability, falls under category C, seats 5 people, and is priced at 35575.


'The characteristics of the Volkswagen ID.3 Pro are as follows:\n\n1. Range: The ID.3 Pro has a range of 340 miles, which means it can travel a significant distance on a single charge.\n\n2. Drive: It is a rear-wheel drive vehicle, which provides good traction and handling.\n\n3. Charging Capability: The ID.3 Pro is equipped with Type 2 CCS charging capability, allowing it to be charged at various charging stations.\n\n4. Category: The ID.3 Pro falls under category C, which indicates its size and classification.\n\n5. Seating Capacity: It has a seating capacity of 5 people, making it suitable for small families or groups.\n\n6. Price: The price of the Volkswagen ID.3 Pro is 35,575, which may vary depending on the specific model and additional features.\n\nThese characteristics make the Volkswagen ID.3 Pro a versatile and efficient electric car option.'

In [5]:
%pip install pyngrok nest_asyncio fastapi uvicorn loguru websockets

Note: you may need to restart the kernel to use updated packages.


In [6]:
from fastapi import FastAPI, WebSocket, WebSocketDisconnect
from fastapi.responses import HTMLResponse
import asyncio
#from bs4 import BeautifulSoup

socket = 8000

app = FastAPI()

'''@app.get("/")
async def get():
    return HTMLResponse(open("index.html").read())

@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    while True:
        data = await websocket.receive_text()
        await websocket.send_text(f"Message received: {data}")
'''
html = open("web/index.html","r").read()

@app.get("/")
async def get():
    return HTMLResponse(html)

@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    while True:
        data = await websocket.receive_text()
        await websocket.send_text(f"ShalopAI: {agent.chat(data).response}")




In [7]:
'''from fastapi import FastAPI
from pydantic import BaseModel
from loguru import logger

app = FastAPI()

class UserRequestIn(BaseModel):
    text: str
    
@app.post("/test")
def index(request: UserRequestIn):
    logger.debug(request)
    return {"ok": True}'''

'from fastapi import FastAPI\nfrom pydantic import BaseModel\nfrom loguru import logger\n\napp = FastAPI()\n\nclass UserRequestIn(BaseModel):\n    text: str\n    \n@app.post("/test")\ndef index(request: UserRequestIn):\n    logger.debug(request)\n    return {"ok": True}'

In [8]:
NGROK_AUTH_TOKEN=os.getenv("NGROK_AUTH_TOKEN")
!ngrok config add-authtoken {NGROK_AUTH_TOKEN}

Authtoken saved to configuration file: C:\Users\79779\AppData\Local/ngrok/ngrok.yml


In [9]:
from pyngrok import ngrok
#register on ngrok.com and get a token here: https://dashboard.ngrok.com/get-started/your-authtoken
# put it then in .env under NGROK_AUTH_TOKEN

ngrok_tunnel = ngrok.connect(socket)

ngrok_tunnel

<NgrokTunnel: "https://a2fd-2001-4ca0-0-f233-e413-8a2b-1659-4a1a.ngrok-free.app" -> "http://localhost:8000">

In [ ]:
import nest_asyncio
import uvicorn

nest_asyncio.apply()
uvicorn.run(app, port=socket)

INFO:     Started server process [7484]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:50245 - "GET / HTTP/1.1" 200 OK


INFO:     ('127.0.0.1', 50246) - "WebSocket /ws" [accepted]
INFO:     connection open


Added user message to memory: lol
